# **1) Import Libraries and get TensorFlow files needed**

In [ ]:
import os
import pathlib
import glob
import cv2 as cv2
import numpy as np
import xml.etree.ElementTree as ET
import pandas as pd

In [ ]:
# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 4065, done.
remote: Counting objects: 100% (4065/4065), done.
remote: Compressing objects: 100% (3089/3089), done.
remote: Total 4065 (delta 1186), reused 1959 (delta 916), pack-reused 0
Receiving objects: 100% (4065/4065), 54.71 MiB | 15.75 MiB/s, done.
Resolving deltas: 100% (1186/1186), done.


In [ ]:
#navigate to /models/research folder to compile protos
%cd models/research

# Compile protos.
!protoc object_detection/protos/*.proto --python_out=.

# Install TensorFlow Object Detection API.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

/content/models/research
Processing /content/models/research
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 74.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 99.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 80.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.5 MB/s eta 0:00:00
     ━━━━

# **2) Mount folder and access images and annotations**

In [ ]:
#First create a shortcut to the shared folder!

from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
#!ln -s /content/gdrive/My\ Drive/ /mydrive
#!ls /mydrive/MathWorks\ \#2\ \(BOS\)\ -\ Classify\ Object\ Behavior\ to\ Enhance\ the\ Safety\ of\ Autonomous\ Vehicles

!ln -s /content/gdrive/My\ Drive/MathWorks\ \#2\ \(BOS\)\ -\ Classify\ Object\ Behavior\ to\ Enhance\ the\ Safety\ of\ Autonomous\ Vehicles/ /myfiles
!ls /myfiles

Mounted at /content/gdrive
'Bridge To Studio'
 Datasets_Samples
'Fall 2023 AI Studio | Final Project Presentation.gslides'
'MathWorks #2 (BOS) - Classify Object Behavior to Enhance the Safety of Autonomous Vehicles'
'Meeting Notes'
 models
 ModelTrainTest
'[Overview Slide] BOS_MathWorks_Classify Object Behavior to Enhance the Safety of Autonomous Vehicles.gslides'
'Presentation Notes.gdoc'
'Project Scope and Deliverables_Fall 2023 AI Studio.gdoc'
 Resources
 SlidesMeetings


In [ ]:
BASEPATH = '/myfiles/ModelTrainTest/'
DATA_BASEPATH = BASEPATH+'data/'

# Parent Directory paths
parent_dir_images = DATA_BASEPATH+'images'
parent_dir_xmls = DATA_BASEPATH+'annotations'

In [ ]:
#Create folders if needed
try:
  os.mkdir(parent_dir_images)
except FileExistsError:
  pass

try:
  os.mkdir(parent_dir_xmls)
except FileExistsError:
  pass

## unzip the datasets and their contents so that they are now in DATA_BASEPATH/data/ folder
# UNCOMMENT BELOW IF YOU NEED TO UNZIP AGAIN, OTHERWISE KEEP IT COMMENTED
!unzip $BASEPATH'images.zip' -d $parent_dir_images
#!unzip $BASEPATH'annotations.zip' -d $parent_dir_xmls


error:  must specify directory to which to extract with -d option
Archive:  /myfiles/ModelTrainTest/annotations.zip
  inflating: /myfiles/ModelTrainTest/data/annotations/n003-2018-01-03-12-03-23+0800__CAM_FRONT__1514952883296507.xml  
  inflating: /myfiles/ModelTrainTest/data/annotations/n003-2018-01-03-12-03-23+0800__CAM_FRONT__1514952943631745.xml  
  inflating: /myfiles/ModelTrainTest/data/annotations/n003-2018-01-03-12-03-23+0800__CAM_FRONT_LEFT__1514952246147171.xml  
  inflating: /myfiles/ModelTrainTest/data/annotations/n003-2018-01-03-12-03-23+0800__CAM_FRONT_LEFT__1514952261647740.xml  
  inflating: /myfiles/ModelTrainTest/data/annotations/n003-2018-01-03-12-03-23+0800__CAM_FRONT_RIGHT__1514952686688255.xml  
  inflating: /myfiles/ModelTrainTest/data/annotations/n003-2018-01-03-14-33-35+0800__CAM_FRONT__1514961595097860.xml  
  inflating: /myfiles/ModelTrainTest/data/annotations/n003-2018-01-03-15-21-35+0800__CAM_FRONT__1514964197520502.xml  
  inflating: /myfiles/ModelTrainTes

# **DO STEPS 3 AND 4 IF YOU NEED TO GENERATE TENSORFLOW RECORDS. IF NOT, SKIP TO STEP 5**

# **3) Create training and testing sets**

In [ ]:
test_labels = DATA_BASEPATH+'test_labels'
train_labels = DATA_BASEPATH+'train_labels'
#Create folders if needed for training and testing labels
try:
  os.mkdir(test_labels)
except FileExistsError:
  pass

try:
  os.mkdir(train_labels)
except FileExistsError:
  pass

%cd $DATA_BASEPATH
# Assuming 500 files:   100 for testing, 400 for training
!ls annotations/* | sort -R | head -100 | xargs -I{} mv {} test_labels/
!ls annotations/* | xargs -I{} mv {} train_labels/

/content/gdrive/.shortcut-targets-by-id/1V6R5dIPvEZICGv8F8ogtd3KU8lk3Bltk/MathWorks #2 (BOS) - Classify Object Behavior to Enhance the Safety of Autonomous Vehicles/ModelTrainTest/data


# **4) Generate label map and tensorflow records**

In [ ]:
def xml_to_csv(path):
  classes_names = []
  xml_list = []

  for xml_file in glob.glob(path + '/*.xml'):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for member in root.findall('object'):
      classes_names.append(member[0].text)
      value = (root.find('filename').text  ,
               int(root.find('size')[0].text),
               int(root.find('size')[1].text),
               member[0].text,
               int(member[4][0].text),
               int(member[4][1].text),
               int(member[4][2].text),
               int(member[4][3].text))
      xml_list.append(value)
  column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
  xml_df = pd.DataFrame(xml_list, columns=column_name)
  classes_names = list(set(classes_names))
  classes_names.sort()
  return xml_df, classes_names

for label_path in ['train_labels', 'test_labels']:
  image_path = os.path.join(os.getcwd(), label_path)
  xml_df, classes = xml_to_csv(label_path)
  xml_df.to_csv(f'{label_path}.csv', index=None)
  print(f'Successfully converted {label_path} xml to csv.')

label_map_path = os.path.join("label_map.pbtxt")
pbtxt_content = ""

for i, class_name in enumerate(classes):
    pbtxt_content = (
        pbtxt_content
        + "item {{\n    id: {0}\n    name: '{1}'\n}}\n\n".format(i + 1, class_name)
    )
pbtxt_content = pbtxt_content.strip()
with open(label_map_path, "w") as f:
    f.write(pbtxt_content)
    print('Successfully created label_map.pbtxt ')

Successfully converted train_labels xml to csv.
Successfully converted test_labels xml to csv.
Successfully created label_map.pbtxt 


In [ ]:
#Note: Current working dir is DATA_BASEPATH
%cd $DATA_BASEPATH

#Usage:
#!python generate_tfrecord.py output.csv output_pb.txt /path/to/images output.tfrecords

path_generate_tf = BASEPATH+'generate_tfrecord.py'

#For train.record
!python $path_generate_tf train_labels.csv  label_map.pbtxt images/ train.record

#For test.record
!python $path_generate_tf test_labels.csv  label_map.pbtxt images/ test.record

/content/gdrive/.shortcut-targets-by-id/1V6R5dIPvEZICGv8F8ogtd3KU8lk3Bltk/MathWorks #2 (BOS) - Classify Object Behavior to Enhance the Safety of Autonomous Vehicles/ModelTrainTest/data
2023-12-03 16:52:15.485304: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-03 16:52:15.531060: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-03 16:52:15.531103: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-03 16:52:15.532302: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-03 16:52:15.539622: I extern

# **5) Get specific model and update TensorFlow before training model**

Downgrade TensorFlow version so we avoid errors in Google Colab

**Note:** When running the cell below you might get a message to restart current runtime. DO NOT restart current runtime, simply move to the next cell.

In [ ]:
#Note: Current working dir is DATA_BASEPATH
%cd $DATA_BASEPATH

# Download the pre-trained model
# Example: Faster R-CNN
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet101_v1_1024x1024_coco17_tpu-8.tar.gz
!tar -xzvf faster_rcnn_resnet101_v1_1024x1024_coco17_tpu-8.tar.gz





/content/gdrive/.shortcut-targets-by-id/1V6R5dIPvEZICGv8F8ogtd3KU8lk3Bltk/MathWorks #2 (BOS) - Classify Object Behavior to Enhance the Safety of Autonomous Vehicles/ModelTrainTest/data
--2023-12-03 17:08:12--  http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet101_v1_1024x1024_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.251.10.207, 142.251.12.207, 172.217.194.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|142.251.10.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 353893393 (337M) [application/x-tar]
Saving to: ‘faster_rcnn_resnet101_v1_1024x1024_coco17_tpu-8.tar.gz’

faster_rcnn_resnet1 100%[===================>] 337.50M  20.7MB/s    in 18s     

2023-12-03 17:08:30 (18.9 MB/s) - ‘faster_rcnn_resnet101_v1_1024x1024_coco17_tpu-8.tar.gz’ saved [353893393/353893393]

faster_rcnn_resnet101_v1_1024x1024_coco17_tpu-8/
faster_rcnn_resnet101_v1_1024x1024_coco17_t

In [ ]:
!pip install tensorflow==2.9

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.7/511.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 99.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 53.9 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 23.5.26
    Uninstalling flatbuffers-23.5.26:
      Successfully uninstalled flatbuffers-23.5.26
  Attempting uninstall: ten

In [ ]:
!pip show tensorflow

Name: tensorflow
Version: 2.9.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, keras-preprocessing, libclang, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine-rl, tensorflow-text, tf-models-official


# **6) Create a new configuration file for the model chosen and run training**

In [ ]:
!cp /content/models/research/object_detection/configs/tf2/faster_rcnn_resnet101_v1_1024x1024_coco17_tpu-8.config $DATA_BASEPATH

base_config_path = DATA_BASEPATH+'faster_rcnn_resnet101_v1_1024x1024_coco17_tpu-8.config'
labelmap_path = DATA_BASEPATH+'label_map.pbtxt'
fine_tune_checkpoint = DATA_BASEPATH+'faster_rcnn_resnet101_v1_1024x1024_coco17_tpu-8/checkpoint/ckpt-0'
train_record_path = DATA_BASEPATH+'train.record'
test_record_path = DATA_BASEPATH+'test.record'
batch_size = 2
num_steps = 8000
num_eval_steps = 1000

import re

with open(base_config_path) as f:
    config = f.read()

with open('model_config.config', 'w') as f:

  # Set labelmap path
  config = re.sub('label_map_path: ".*?"',
             'label_map_path: "{}"'.format(labelmap_path), config)

  # Set fine_tune_checkpoint path
  config = re.sub('fine_tune_checkpoint: ".*?"',
                  'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), config)

  # Set train tf-record file path
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")',
                  'input_path: "{}"'.format(train_record_path), config)

  # Set test tf-record file path
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")',
                  'input_path: "{}"'.format(test_record_path), config)

  # Set number of classes (CHECK LABEL MAP FILE TO UPDATE THIS NUMBER)
  config = re.sub('num_classes: [0-9]+',
                  'num_classes: {}'.format(6), config)

  # Set batch size
  config = re.sub('batch_size: [0-9]+',
                  'batch_size: {}'.format(batch_size), config)

  # Set training steps
  config = re.sub('num_steps: [0-9]+',
                  'num_steps: {}'.format(num_steps), config)

  # Set fine-tune checkpoint type to detection
  config = re.sub('fine_tune_checkpoint_type: "classification"',
             'fine_tune_checkpoint_type: "{}"'.format('detection'), config)

  f.write(config)

In [ ]:
%cat model_config.config

# Faster R-CNN with Resnet-101 (v1),
# w/high res inputs, long training schedule
# Trained on COCO, initialized from Imagenet classification checkpoint
#
# Train on TPU-8
#
# Achieves 37.1 mAP on COCO17 val

model {
  faster_rcnn {
    num_classes: 6
    image_resizer {
      fixed_shape_resizer {
        width: 1024
        height: 1024
      }
    }
    feature_extractor {
      type: 'faster_rcnn_resnet101_keras'
      batch_norm_trainable: true
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 16
        width_stride: 16
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        truncated_normal_initializer {
          stddev: 0.01
        }
      }
    }
    first_stage_nms_score_threshold: 0.0
    first_stage_nms_iou_threshold: 0.7

Start TensorBoard

In [ ]:
# Update the directories as you see fit
model_dir = BASEPATH+'training/'
pipeline_config_path = DATA_BASEPATH+'model_config.config'

model_dir

'/myfiles/ModelTrainTest/training/'

In [ ]:
# Run this block first and use the refresh arrow that will appear in the header once the training below starts
%reload_ext tensorboard
%tensorboard --logdir '/myfiles/ModelTrainTest/training/' --load_fast true

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
Address already in use
Port 6006 is in use by another program. Either identify and stop that program, or start the server with a different port.

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_config_path} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps={num_eval_steps}

/usr/local/lib/python3.10/dist-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/usr/local/lib/python3.10/dist-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/usr/local/lib/python3.10/dist-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl5mutexC1Ev']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/usr/local/lib/python3.10/dist-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/usr/local/lib/python3.10/dist-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/usr/local/lib/python3.10/dist-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZNK10tensorflow4data11DatasetBase8FinalizeEPNS_15OpKernelContextESt8functionIFN3tsl8StatusOrINS5_4

# **Generate inference graph**

In [ ]:
output_directory = BASEPATH+'inference'

!python /content/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir {model_dir} \
    --output_directory {output_directory} \
    --pipeline_config_path {pipeline_config_path}